<a id=top> </a>
# Monitoring job runs

**Author: Christophe Zutterman** <br>
*Date of creation: 14/01/2022* <br>

Documentation: Notebook to check CP4D job runs for failed status or jobs runs that are stuck in 'Starting' status. A mail is sent to the stakeholders if these kind of issues have been detected <br>

Program flow: <br>
[0. Import modules and declare vars/tokens/functions/... ](#impmod) <br>
[1. Define parameters](#defpar) <br>
[2. Get projects](#getpro) <br>
[3. Get jobs from projects](#getjob) <br>
[4. Send alert in case of failed jobs](#sendalert) <br>

[Z. Clean workspace](#clewor) <br>

*History of notebook changes :*<br>
* 2022/01/04 (by Christophe Zutterman) : timeframe delay in seconds adjusted from 10 -> 60<br>
* 2022/03/17 (by Anja Clompen) : Workaround JIRA BDAP-556 in order to fetch project_name<br>
* 2022/05/02 (by Anja Clompen) : Adding "Execution of some commands facilitating Deloitte Service Desk error tracing"  (Deloitte ticket BDAP-572)<br>
* 2023/03/31 (by Dries Van Dromme) : Adaptations to CP4D 4.6.2 (CAB-65) denoted with "# CP4D 4.6.2 Adaptations"*<br>

*Execution of some commands facilitating Deloitte Service Desk error tracing*

In [2]:
# Do not run - Keep for future reference
# CP4D 3.5
#ls -AlF /opt/ibm/ws/bin/

total 52
-rwxr-xr-x. 1 wsuser watsonstudio  2628 Mar 11 21:20 create_certificate.sh*
-rwxr-xr-x. 1 wsuser watsonstudio  3157 Mar 11 21:20 custom_conda.sh*
-rwxr-xr-x. 1 wsuser watsonstudio   769 Mar 11 21:20 init_terminal.sh*
-rwxr-xr-x. 1 wsuser watsonstudio  3678 Mar 11 21:20 kernel-setup.sh*
-rwxr-xr-x. 1 wsuser watsonstudio  1000 Mar 11 21:20 print-env.sh*
-rwxr-xr-x. 1 wsuser watsonstudio  1293 Mar 11 21:20 run_job.sh*
-rw-r--r--. 1 wsuser watsonstudio  9636 Mar 11 21:20 run-notebook-job.py
-rwxr-xr-x. 1 wsuser watsonstudio 13318 Mar 11 21:20 setup_container.sh*
-rwxr-xr-x. 1 wsuser watsonstudio  6601 Mar 11 21:20 setup_git_repository.sh*
-rwxr-xr-x. 1 wsuser watsonstudio  7978 Mar 11 21:20 start_jupyter.sh*
-rwxr-xr-x. 1 wsuser watsonstudio  1474 Mar 11 21:20 start_status.sh*


In [1]:
ls -AlF /opt/ibm/ws/bin/

total 85
-rwxr-xr-x. 1 1000690000 wscommon   496 Mar  6 10:37 bash-setup.sh*
-rwxr-xr-x. 1 1000690000 wscommon  3227 Mar  6 10:37 create_certificate.sh*
-rwxr-xr-x. 1 1000690000 wscommon  3884 Mar  6 10:37 custom_conda.sh*
-rw-r--r--. 1 1000690000 wscommon  1170 Mar  6 10:37 get-proxy-from-condarc.py
-rwxr-xr-x. 1 1000690000 wscommon  1118 Mar  6 10:37 init_terminal.sh*
-rwxr-xr-x. 1 1000690000 wscommon  4490 Mar  6 10:37 kernel-setup.sh*
-rwxr-xr-x. 1 1000690000 wscommon  1000 Mar  6 10:37 print-env.sh*
-rwxr-xr-x. 1 1000690000 wscommon  1557 Mar  6 10:37 run_job.sh*
-rw-r--r--. 1 1000690000 wscommon 11593 Mar  6 10:37 run-notebook-job.py
-rwxr-xr-x. 1 1000690000 wscommon 19120 Mar  6 10:37 setup_container.sh*
-rwxr-xr-x. 1 1000690000 wscommon  8028 Mar  6 10:37 setup_git_repository.sh*
-rwxr-xr-x. 1 1000690000 wscommon  1295 Mar  6 10:37 setup_sshenv.sh*
-rwxr-xr-x. 1 1000690000 wscommon   941 Mar  6 10:37 shutdown.sh*
-rwxr-xr-x. 1 1000690000 wscommon  8958 Mar  6 10:37 start_jupyte

In [3]:
# Do not run - Keep for future reference
#cat /opt/ibm/ws/bin/run_job.sh

#!/bin/bash

# Licensed Materials - Property of IBM
# 5737-C49
# 5737-B37
# 5737-D37
# (C) Copyright IBM Corp. 2017, 2019    All Rights Reserved.
# US Government Users Restricted Rights - Use, duplication or disclosure
# restricted by GSA ADP Schedule Contract with IBM Corp.

#export APP_ENV_TORNADO_VALIDATE_CERT_HTTP_CLIENTS=False
#export DSX_JUPYTER_BASE_ROUTE=dsx-jupyter-py36
source /opt/ibm/ws/bin/setup_container.sh &

if [ -z ${DSX_USER_ID} ];
then
  export DSX_USER_ID=${USER}
fi
export APP_ENV_CDSX_NOTEBOOKS_API=${APP_ENV_NOTEBOOKS_API_V2}
export APP_ENV_AX_PROJECTS_API=${APP_ENV_APSX_API}
export DSX_PROJECT_DIR=$HOME
system_ca_certs=/etc/pki/ca-trust/extracted/openssl/ca-bundle.trust.crt
    if [[ -e $system_ca_certs ]]; then
        # The conda environments have their own certificate collections.
        # Instead of adding WS Local certificates to all of them, set
        # environment variables pointing to the system certificates.
        # SSL_CERT_FILE for OpenSSL, REQUESTS

In [2]:
cat /opt/ibm/ws/bin/run_job.sh

#!/bin/bash

# Licensed Materials - Property of IBM
# 5737-C49
# 5737-B37
# 5737-D37
# (C) Copyright IBM Corp. 2017, 2019    All Rights Reserved.
# US Government Users Restricted Rights - Use, duplication or disclosure
# restricted by GSA ADP Schedule Contract with IBM Corp.

#export APP_ENV_TORNADO_VALIDATE_CERT_HTTP_CLIENTS=False
#export DSX_JUPYTER_BASE_ROUTE=dsx-jupyter-py36

_basedir_=$(realpath $(dirname $0)/..)
_ax_ext_lib="${_basedir_}/ax-ext/lib"
if [[ -d ${_ax_ext_lib} && ":$PYTHONPATH:" != *":${_ax_ext_lib}:"* ]]; then
    export PYTHONPATH="${_ax_ext_lib}${PYTHONPATH:+:$PYTHONPATH}"
fi

source /opt/ibm/ws/bin/setup_container.sh &

source activate ${DSX_JUPYTER_CONDENV}

if [ -z ${DSX_USER_ID} ];
then
  export DSX_USER_ID=${USER}
fi
export APP_ENV_CDSX_NOTEBOOKS_API=${APP_ENV_NOTEBOOKS_API_V2}
export APP_ENV_AX_PROJECTS_API=${APP_ENV_APSX_API}
export DSX_PROJECT_DIR=$HOME
system_ca_certs=/etc/pki/ca-trust/extracted/openssl/ca-bundle.trust.crt
    if [[ -e $system_ca_certs ]]

*Print program flow to log*

In [3]:
print('''
General program flow:
    0. Import modules and declare vars/tokens/functions/...
    1. Define parameters
    2. Get projects
    3. Get jobs from projects
    4. Send alert in case of failed jobs
 
    Z. Clean workspace''')


General program flow:
    0. Import modules and declare vars/tokens/functions/...
    1. Define parameters
    2. Get projects
    3. Get jobs from projects
    4. Send alert in case of failed jobs
 
    Z. Clean workspace


*Print user info and run info to log*

In [4]:
import os
import requests
import datetime
token = os.environ['USER_ACCESS_TOKEN']
header={'Authorization':'Bearer ' + token}

In [ ]:
# CP4D 3.5
# Get username of user running the job
#apiurl_connection='https://bdap.socialsecurity.be/icp4d-api/v1/me'
#fetchuser=requests.get(apiurl_connection,headers=header)
#userrunning=fetchuser.json()['UserInfo']['displayName']

In [5]:
# CP4D 3.6.2 Adaptations
# Get username of user running the job
apiurl_connection='https://bdap.socialsecurity.be/usermgmt/v1/user/currentUserInfo'
fetchuser=requests.get(apiurl_connection,headers=header)
userrunning=fetchuser.json()['display_name']

In [ ]:
# CP4D 3.5 - Workaround JIRA BDAP-556
#project_id=os.environ['PROJECT_ID']
#apiurl_connection='https://bdap.socialsecurity.be/v2/projects/' + project_id
#  #fetchname=requests.get(apiurl_connection,headers=header, verify=False) #don't do this
#fetchname=requests.get(apiurl_connection,headers=header)
#project_name=fetchname.json()['entity']['name']

In [6]:
# CP4D 4.6.2 Adaptations
# Standard way of doing things, with ibm_watson_studio_lib
# https://www.ibm.com/docs/en/cloud-paks/cp-data/4.6.x?topic=uwsl-migrating-from-project-lib-python-watson-studio-lib
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()
project_id = wslib.here.get_ID()
project_name = wslib.here.get_name()

In [7]:
# Print general user/project/date time of run to log
# project_id=os.environ['PROJECT_ID']
# print("#### Starting from CP4D project:",os.environ['PROJECT_NAME'])
print("#### Starting from CP4D project:",project_name)
print("#### Project ID:",project_id)
print("#### User running job/script/notebook:", userrunning)
print("#### Start run date/time:",datetime.datetime.now())

#### Starting from CP4D project: DEV - Monitoring, Reporting, Alerting
#### Project ID: 922eef5c-ed37-4c82-9e12-b1fc2c343196
#### User running job/script/notebook: Dries Van Dromme
#### Start run date/time: 2023-05-16 13:26:20.143228


<a id=impmod> </a>
# 0. Import modules and declare vars/tokens/functions/...
[gototop](#top)

In [8]:
print("#### 0. Import modules and declare vars/tokens/functions/...",datetime.datetime.now())

#### 0. Import modules and declare vars/tokens/functions/... 2023-05-16 13:26:56.740708


In [9]:
import pandas as pd
import smtplib

<a id=defpar> </a>
# 1. Define parameters
[gototop](#top)

In [10]:
print("#### 1. Define parameters",datetime.datetime.now())

#### 1. Define parameters 2023-05-16 13:27:06.659332


In [11]:
# Declaring header of email message
email_message="The following jobs have issues. Either they have failed or are stuck in 'Starting' status for more than 2 minutes: \r\n"

In [12]:
# Setting error handling constants
error_state=0   # if an error is detected during the process, the error_state value is increased
failed_job_state=0  # if a failed job has been detected, the failed_job_state is increased

In [13]:
# If run_every_x_hour set to 1 then the jobruns of the last hour will be checked. If set to 2 the jobruns of the last 2 hours will be checked.
run_every_x_hour=1

In [14]:
# Setting the timeframe within which the jobruns will be checked
d_event_start=datetime.datetime.now() - datetime.timedelta(hours=run_every_x_hour, seconds=60)  # 20220104 delay in seconds adjusted from 10 -> 60 by Christophe Zutterman
d_event_end=d_event_start+ datetime.timedelta(hours=run_every_x_hour, seconds=60)               # 20220104 delay in seconds adjusted from 0 -> 60 by Christophe Zutterman

In [15]:
# CP4D 4.6.2 Adapatations
# Parameter governing whether to monitor the PRD-jobs or rather the DEV-jobs
monitor_env_jobs="DEV" # in order to monitor the DEV-jobs
#monitor_env_jobs="PRD" # in order to monitor the PRD-jobs

<a id=getpro> </a>
# 2. Get projects
[gototop](#top)

In [16]:
print("#### 2. Get projects",datetime.datetime.now())

#### 2. Get projects 2023-05-16 13:27:14.540942


In [17]:
# Get CP4D projects 
project_iterator=0
params={'limit':100,'skip':0}
apiurl_connection='https://bdap.socialsecurity.be/v2/projects/'
fetchprojects=requests.get(apiurl_connection,headers=header,params=params)
df_projects=pd.json_normalize(fetchprojects.json(),'resources')
while fetchprojects.json()['total_results'] > 0:
    project_iterator+=1
    params={'limit':100,'skip':project_iterator*100}
    apiurl_connection='https://bdap.socialsecurity.be/v2/projects/'
    fetchprojects=requests.get(apiurl_connection,headers=header,params=params)  
    if fetchprojects.json()['total_results'] > 0:
        df_projects_delta=pd.json_normalize(fetchprojects.json(),'resources')
        df_projects=pd.concat([df_projects,df_projects_delta])    

In [ ]:
# CP4D 3.5
# Select projects starting with 'PRD' or 'PRD - '
#df_projects=df_projects.loc[df_projects['entity.name'].str.upper().str.startswith(('PRD ','PRD-'))]

In [18]:
# CP4D 4.6.2 Adaptations
# Select DEV or PRD jobs by their names starting with 'DEV( )(-)' or 'PRD( )(-)'
if monitor_env_jobs=='PRD':
    # Select projects starting with 'PRD' or 'PRD - '
    df_projects=df_projects.loc[df_projects['entity.name'].str.upper().str.startswith(('PRD ','PRD-'))]
    print('+++monitoring PRD jobs+++')
elif monitor_env_jobs=='DEV':
    # Select projects starting with 'DEV' or 'DEV - '
    df_projects=df_projects.loc[df_projects['entity.name'].str.upper().str.startswith(('DEV ','DEV-'))]
    print('+++monitoring DEV jobs+++')
else:
    error_message='Error in parameter monitor_env_job: it should be either "PRD" or "DEV"'
    error_state=1

+++monitoring DEV jobs+++


In [19]:
# Get project ids and put them in a list
if df_projects.shape[0]>0:
    list_project_ids=df_projects['metadata.guid'].values.tolist()
    print("#### Number of projects found: ",len(list_project_ids))
else:
    error_message='No projects found'
    error_state=1

#### Number of projects found:  22


In [20]:
list_project_names=df_projects['entity.name'].values.tolist()

In [47]:
# Do not run - Keep for reference
#list_project_names

['DEV - Monitoring, Reporting, Alerting',
 'DEV - API FUNCTIONS',
 'DEV - INVESTIGATIONS INSPECTIE2020',
 'DEV - Scoring ABT Belgian employer',
 'DEV - Templates',
 'DEV - CP4D RSZ Health Check',
 'DEV - POC FOD FIN',
 'DEV - Able/Willing supervised model',
 'DEV - Deeltijdse Arbeid Supervised Model',
 'DEV - Human Trafficking Supervised Model',
 'DEV - Training ABT Belgian Company',
 'DEV - MiningModels Training',
 'DEV - Dimona supervised model',
 'DEV - CP4D RSZ Testscripts',
 'DEV - AD HOC 2021',
 'DEV - AD HOC 2022',
 'DEV - SocDump_v1.0',
 'DEV - MDM_ref',
 'DEV - Data Quality Monitoring',
 'DEV - Feedback datamining results']

In [21]:
list_project_names
#Note: New: Population enterprises and employers + CP4D-RSZ-Datastagetests

['DEV - SocDump_v1.0',
 'DEV - Deeltijdse Arbeid Supervised Model',
 'DEV - MiningModels Training',
 'DEV - Scoring ABT Belgian employer',
 'DEV - Feedback datamining results',
 'DEV - INVESTIGATIONS INSPECTIE2020',
 'DEV - API FUNCTIONS',
 'DEV - Training ABT Belgian Company',
 'DEV - AD HOC 2022',
 'DEV - MDM_ref',
 'DEV - CP4D RSZ Health Check',
 'DEV - AD HOC 2021',
 'DEV - Monitoring, Reporting, Alerting',
 'DEV-CP4D-RSZ-DatastageTests',
 'DEV - CP4D RSZ Testscripts',
 'DEV - Templates',
 'DEV - Population enterprises and employers',
 'DEV - POC FOD FIN',
 'DEV - Able/Willing supervised model',
 'DEV - Data Quality Monitoring',
 'DEV - Dimona supervised model',
 'DEV - Human Trafficking Supervised Model']

<a id=getjob> </a>
# 3. Get jobs from projects
[gototop](#top)

In [22]:
print("#### 3. Get jobs from projects",datetime.datetime.now())

#### 3. Get jobs from projects 2023-05-16 13:32:06.720439


In [18]:
# Do not run - Keep for reference
# contains output of next cell if monitor_env_jobs=='PRD'

## Project name:  PRD - Scoring ABT Belgian employer
#### Number of jobs found:  2
## Project name:  PRD - Test Technical User
#### Number of jobs found:  2
## Project name:  PRD - Training ABT Belgian Company
#### Number of jobs found:  1
## Project name:  PRD - INVESTIGATIONS INSPECTIE2020
#### Number of jobs found:  1
## Project name:  PRD - CP4D RSZ Testscripts
#### Number of jobs found:  10
####  Project: PRD - CP4D RSZ Testscripts: job ['Modeling_Predicting - JEG Spark'] issues at ['2022-01-25T08:13:35Z']

## Project name:  PRD - API FUNCTIONS
#### Number of jobs found:  10
## Project name:  PRD - Able/Willing
#### Number of jobs found:  18


In [63]:
# CP4D 4.6.2 Adaptations
# https://cloud.ibm.com/apidocs/machine-learning-cp > Methods > Deployment Jobs > Retrieve the Deployment Jobs:
# Retrieve the status of the current jobs. The system will apply a max limit of jobs retained by the system as we cannot accumulate an infinite number of jobs.
# Only most recent 300 jobs (system configurable) will be preserved. Older jobs will be purged by the system.
# limit   Always included*   integer   The number of items to return in each page.
#         Possible values: 1 ≤ value ≤ 200
# --> eventually it turns out that limit max = 200 in
#     params={'project_id':list_project_ids[i_projectid],'limit':10000}
#     apiurl_connection='https://bdap.socialsecurity.be/v2/jobs/'+list_jobs_ids[i_jobid]+'/runs'
#     fetchjobruns=requests.get(apiurl_connection,headers=header,params=params)
# 16/03/20233: added try/except & jobs_found conditions
jobs_found=True
if error_state==0:
    # Iterate over all found projects
    for i_projectid in range(len(list_project_ids)):
        print("## Project name: ",list_project_names[i_projectid])

        # Get jobs from the projects
        params={'project_id':list_project_ids[i_projectid]}
        apiurl_connection='https://bdap.socialsecurity.be/v2/jobs/'
        fetchjobs=requests.get(apiurl_connection,headers=header,params=params)
        #df_jobs=pd.json_normalize(fetchjobs.json(),'results')
        try:
            df_jobs=pd.json_normalize(fetchjobs.json(),'results')
        except KeyError:
            #print('There is a KeyError; the "results" key is expected to be part of the fetchjobs.json() response.')
            if fetchjobs.json()['code']==403:
                #print('This is not the case. The Response code is 403 \n (Forbidden: You are not permitted to perform this action. See response for more information.)')
                #print(fetchjobs.json()['reason'])
                print('Permission Denied: Authenticated user is not a member of the project.\r\n')
                jobs_found=False
            else:
                raise KeyError('There is a KeyError; the "results" key is expected to be part of the fetchjobs.json() response. This is not the case. \nFurthermore, the reason is not 403 "Permission denied. Authenticated user is not a member of the project."')

        # If jobs have been found, get job ids and put them in a list
        if jobs_found==True and df_jobs.shape[0]>0:
            list_jobs_ids=df_jobs['metadata.asset_id'].values.tolist()
            print("#### Number of jobs found: ",len(list_jobs_ids))

            # Iterate over all found jobs
            for i_jobid in range(len(list_jobs_ids)):
                # Get all jobruns for the job
                #CP4D 3.5
                #params={'project_id':list_project_ids[i_projectid],'limit':10000}
                #CP4D 4.6.2
                params={'project_id':list_project_ids[i_projectid],'limit':200}
                apiurl_connection='https://bdap.socialsecurity.be/v2/jobs/'+list_jobs_ids[i_jobid]+'/runs'
                fetchjobruns=requests.get(apiurl_connection,headers=header,params=params)
                
                # If jobruns have been found, get info of failed ones withing the timeframe
                if fetchjobruns.json()['total_rows']>0:
                    df_jobruns=pd.json_normalize(fetchjobruns.json(),'results')

                    # Select failed jobs within timeframe, or jobs that are stuck for more than 2 minutes in Starting status
                    df_jobruns=df_jobruns.loc[(df_jobruns['entity.job_run.state']=='Failed') | ((df_jobruns['entity.job_run.state']=='Starting') & ((d_event_end - df_jobruns['metadata.created_at'].astype('datetime64')).dt.total_seconds()>120 ))]
                    df_jobruns=df_jobruns.loc[(df_jobruns['metadata.created_at'].astype('datetime64')>d_event_start) & (df_jobruns['metadata.created_at'].astype('datetime64')<d_event_end)]

                    # If failed jobsruns are found withing timeframe, get name and timestamp of failed job run
                    if df_jobruns.shape[0]>0:
                        failed_job='Project: ' + str(list_project_names[i_projectid]) + ': job ' + str(df_jobruns['entity.job_run.job_name'].head(1).values) + ' issues at ' + str(df_jobruns['metadata.created_at'].head(1).values) + '\r\n'
                        print("#### ",failed_job)
                        email_message=email_message + '\n' + failed_job
                        failed_job_state+=1

## Project name:  DEV - SocDump_v1.0
## Project name:  DEV - Deeltijdse Arbeid Supervised Model
## Project name:  DEV - MiningModels Training
## Project name:  DEV - Scoring ABT Belgian employer
## Project name:  DEV - Feedback datamining results
## Project name:  DEV - INVESTIGATIONS INSPECTIE2020
## Project name:  DEV - API FUNCTIONS
#### Number of jobs found:  11
## Project name:  DEV - Training ABT Belgian Company
## Project name:  DEV - AD HOC 2022
## Project name:  DEV - MDM_ref
#### Number of jobs found:  1
## Project name:  DEV - CP4D RSZ Health Check
#### Number of jobs found:  5
## Project name:  DEV - AD HOC 2021
## Project name:  DEV - Monitoring, Reporting, Alerting
#### Number of jobs found:  3
####  Project: DEV - Monitoring, Reporting, Alerting: job ['JOB - Monitoring job runs CP4D'] issues at ['2023-05-16T13:20:06Z']

## Project name:  DEV-CP4D-RSZ-DatastageTests
Permission Denied: Authenticated user is not a member of the project
## Project name:  DEV - CP4D RSZ Tests

<a id=sendalert> </a>
# 4. Send alert in case of failed jobs
[gototop](#top)

In [19]:
print("#### 4. Send alert in case of failed jobs",datetime.datetime.now())

#### 4. Send alert in case of failed jobs 2022-01-25 08:58:55.073656


In [148]:
# CP4D 4.6.2 Adaptations
if error_state==0 and failed_job_state>0:
    print("#### Failed jobs found, alert sent. Number of failed jobs:",failed_job_state)
    sender = 'CP4D@onssrszlss.fgov.be'
    receivers = ['dries.vandromme@onssrszlss.fgov.be']
    #receivers = ['anja.clompen@onssrszlss.fgov.be','erna.vandevreken@onssrszlss.fgov.be','dries.vandromme@onssrszlss.fgov.be','olivier.uyttenhove@onssrszlss.fgov.be']

    message = """from: CP4D@onssrszlss.fgov.be
    to: dries.vandromme@onssrszlss.fgov.be
    Subject: CP4D-jobs failed

    {}
    """.format(email_message)

    try:
       smtpObj = smtplib.SMTP('bdapaccessprd01.mgmt',25)
       smtpObj.sendmail(sender, receivers, message)         
       print("Successfully sent email")
    except SMTPException:
       print("Error: unable to send email")

#### Failed jobs found, alert sent. Number of failed jobs: 1
Successfully sent email


In [21]:
if error_state==0 and failed_job_state==0:
    print("#### No failed jobs found.")

<a id=clewor> </a>
# Z. Clean Workspace
[gototop](#top)

In [22]:
print("#### Z. Clean Workspace",datetime.datetime.now())

#### Z. Clean Workspace 2022-01-25 08:59:02.176790


In [23]:
del(email_message, error_state, failed_job_state,run_every_x_hour,d_event_start,d_event_end)

In [24]:
print("#### End run date/time = ",datetime.datetime.now())

#### End run date/time =  2022-01-25 08:59:04.978612
